In [1]:
import pandas as pd
import numpy as np
import random
import sys
sys.path.append('../')
from modules import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
SEED = constants.SEED
random.seed(SEED)
np.random.seed(SEED)

#### Noisiness function

In [3]:
def insert_noise_old(df, frac):   
    no_lupus_indices = df[df.label==0].index.tolist()
    lupus_indices = df[df.label==1].index.to_list()
    no_lupus_to_change_indices = random.sample(no_lupus_indices, int(frac/2*len(no_lupus_indices)))
    lupus_to_change_indices = random.sample(lupus_indices, int(frac/2*len(lupus_indices)))
    df.loc[no_lupus_to_change_indices, 'label'] = 1
    df.loc[lupus_to_change_indices, 'label'] = 0
    return df    

In [4]:
def insert_noise(df, col, frac, new_vals_list):
    non_null_indices = df[df[col].notna()].index.tolist() #getting not null indices
    to_change_indices = random.sample(non_null_indices, int(frac*len(non_null_indices)))
    new_values = np.random.choice(new_vals_list, len(to_change_indices))
    df.loc[to_change_indices, col] = new_values
    return df    

In [5]:
training_df.columns

NameError: name 'training_df' is not defined

#### The data

In [10]:
df = pd.read_csv('../data/orig/lupus_dataset.csv')
df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
1,1,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
3,1,1,0,1,0,0,0,0,0,0,...,1,0,0,0,0,1,1,0,0,1
4,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


#### delete from here

#### end here

In [11]:
class_dict = {'No lupus':0, 'Lupus':1, 'Inconclusive diagnosis':2}
df['label'] = df['label'].replace(class_dict)
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.20, stratify=y, random_state=SEED)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.10, stratify=y_train_val, random_state=SEED)
training_df = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
validation_df = pd.concat([X_val, y_val], axis=1).reset_index(drop=True)
testing_df = pd.concat([X_test, y_test], axis=1).reset_index(drop=True)
# X_train, y_train = np.array(X_train), np.array(y_train)
# X_val, y_val = np.array(X_val), np.array(y_val)
# X_test, y_test = np.array(X_test), np.array(y_test)
# X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

In [12]:
frac = 0.6

In [13]:
for column in training_df.columns:
    if column not in ['ana', 'label']:
        training_df = insert_noise(training_df, column, frac, training_df[column].unique().tolist())
training_df.to_csv(f'../data/noisiness/{frac}/training_set.csv', index=False)

In [14]:
validation_df.to_csv(f'../data/noisiness/{frac}/validation_set.csv', index=False)
testing_df.to_csv(f'../data/noisiness/{frac}/testing_set.csv', index=False)